<div class="alert alert-success h1" role="alert">
    <b>Guardant el model!</b>
</div>

In [2]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler, LabelBinarizer, OrdinalEncoder
from sklearn.linear_model import LogisticRegression

from classes.customTransformers import CustomTransformer

<div class="alert alert-success" role="alert">
    <h2>1. Crear pipeline del model</h2>
</div>

In [4]:
df = pd.read_csv('../Capitol1/train.csv', index_col='PassengerId')
df = df[['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'Survived']]
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
PassengerId,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


<div class="alert alert-warning" role="alert">
    <h3>1.1 Preprocessing</h3>
</div>

In [6]:
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']
numeric_transformer = Pipeline(
    [
        ('imputer_num', SimpleImputer(strategy = 'median')),
        ('robust', RobustScaler())
    ]
)

numeric_features_minmax = ['Pclass']
numeric_transformer_minmax = Pipeline(
    [
        ('imputer_num', SimpleImputer(strategy = 'median')),
        ('minmax', MinMaxScaler())
    ]
)

categorical_features_bin = ['Sex']
categorical_transformer_bin = Pipeline(
    [
        ('imputer_cat', SimpleImputer(strategy = 'constant', fill_value = 'missing')),
        ('ordinalEncoder', OrdinalEncoder())
    ]
)

categorical_features = ['Embarked']
categorical_transformer = Pipeline(
    [
        ('imputer_cat', SimpleImputer(strategy = 'most_frequent', fill_value = 'missing')),
        ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
    ]
)

In [7]:
preprocessor = ColumnTransformer(
    [
        ('name', CustomTransformer(),'Name'),
        ('categoricals', categorical_transformer, categorical_features),
        ('categorical_bin', categorical_transformer_bin, categorical_features_bin),
        ('numericals', numeric_transformer, numeric_features),
        ('numericals_minmax', numeric_transformer_minmax, numeric_features_minmax)
        
    ],
    remainder = 'drop'
)

<div class="alert alert-warning" role="alert">
    <h3>1.2 Pipeline</h3>
</div>

In [8]:
pipeline = Pipeline(
    [
        ('preprocessing', preprocessor),
        ('model', LogisticRegression(C= 0.5, penalty='l2', random_state= 42, solver= 'liblinear'))
    ]
)
pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('name', CustomTransformer(),
                                                  'Name'),
                                                 ('categoricals',
                                                  Pipeline(steps=[('imputer_cat',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked']),
                                                 ('categorical_bin',
                                                  Pipeline(steps=[('imputer_cat',
                                                                   SimpleImputer(fill_value...
                                                                   OrdinalEncoder())]),
                                                  ['Sex']),
                                                 ('numericals',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('robust',
                                                                   RobustScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('numericals_minmax',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('minmax',
                                                                   MinMaxScaler())]),
                                                  ['Pclass'])])),
                ('model',
                 LogisticRegression(C=0.5, random_state=42,
                                    solver='liblinear'))])

<div class="alert alert-success" role="alert">
    <h2>2. Entrenar la pipeline amb totes les dades</h2>
</div>

In [9]:
X = df.drop('Survived', axis = 1)
y = df.Survived

In [10]:
pipeline.fit(X,y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('name', CustomTransformer(),
                                                  'Name'),
                                                 ('categoricals',
                                                  Pipeline(steps=[('imputer_cat',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked']),
                                                 ('categorical_bin',
                                                  Pipeline(steps=[('imputer_cat',
                                                                   SimpleImputer(fill_value...
                                                                   OrdinalEncoder())]),
                                                  ['Sex']),
                                                 ('numericals',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('robust',
                                                                   RobustScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('numericals_minmax',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('minmax',
                                                                   MinMaxScaler())]),
                                                  ['Pclass'])])),
                ('model',
                 LogisticRegression(C=0.5, random_state=42,
                                    solver='liblinear'))])

In [11]:
import joblib
joblib.dump(pipeline, 'best_model.pkl')

['best_model.pkl']

<div class="alert alert-success" role="alert">
    <h2>3. Comprovació del model</h2>
</div>

In [1]:
import joblib
from classes.customTransformers import CustomTransformer

pipeline_loaded = joblib.load('best_model.pkl')

In [3]:
df = pd.read_csv('test.csv', index_col='PassengerId')
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df['Survived'] = pipeline_loaded.predict(df)

In [5]:
df['Survived'].to_csv('out.csv')